In [ ]:
#https://medium.com/@admantium/llm-fine-tuning-workshop-improve-linguistic-skills-0db7a48f4ba8#!pip install torch
#!poetry init --quit
#!poetry add transformers@4.37.2 datasets@2.17.0 jupyter@1.0.0 evaluate@0.4.1 scikit-learn@1.4.1.post1 torch@2.2.0
#!pip install torch  #anaconda -y
#!pip3 install  transformers==4.37.2 
#!pip install accelerate==0.28.0

In [2]:
from transformers import pipeline

model_name = 'openai-community/gpt2'
model = pipeline(
    "question-answering", model=model_name
)
query = {
 "question": "What is NASA?",
 "context": '''
 The National Aeronautics and Space Administration (NASA) is an independent
 agency of the U.S. federal government responsible for the civil space
 program, aeronautics research, and space research. Established in 1958, it
 succeeded the National Advisory Committee for Aeronautics (NACA) to give
 the U.S. space development effort a distinctly civilian orientation,
 emphasizing peaceful applications in space science.[4][5][6] It has since
 led most American space exploration, including Project Mercury, Project
 Gemini, the 1968–1972 Apollo Moon landing missions, the Skylab space
 station, and the Space Shuttle. It currently supports the International
 Space Station and oversees the development of the Orion spacecraft and the
 Space Launch System for the crewed lunar Artemis program, the Commercial
 Crew spacecraft, and the planned Lunar Gateway space station.
 NASA's science is focused on better understanding Earth through the Earth
 Observing System;[7] advancing heliophysics through the efforts of the
 Science Mission Directorate's Heliophysics Research Program;[8] exploring
 bodies throughout the Solar System with advanced robotic spacecraft such as
 New Horizons and planetary rovers such as Perseverance;[9] and researching
 astrophysics topics, such as the Big Bang, through the James Webb Space
 Telescope, the Great Observatories and associated programs.[10] The Launch
 Services Program oversees launch operations and countdown management for
 its uncrewed launches.
 '''
}
answer = model(query)
print(answer)

/Applications/Utilities/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.008792610839009285, 'start': 419, 'end': 422, 'answer': '.[4'}


In [3]:
from datasets import load_dataset
import datasets
data = load_dataset('glue','cola')


In [4]:
print(data['train'][0])
# {'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'label': 1, 'idx': 0}

print(data['train'][42]['sentence'])
# {'sentence': 'They made him to exhaustion.', 'label': 0, 'idx': 42}

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'label': 1, 'idx': 0}
They made him to exhaustion.


In [5]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [6]:
def tokenize_dataset(dataset):
  return tokenizer(dataset['sentence'], truncation=True , padding='do_not_pad')#'max_length', )


In [7]:
tokenized_data = data.map(tokenize_dataset, batched=True)
print(tokenized_data['train'][0])
# {'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'label': 1, 'idx': 0, 'input_ids': [5122, 2460, 1839, 470, 2822, 428, 3781, 11, 1309, 3436, 262, 1306, 530, 356, 18077, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="gpt2_linguistic_finetuning")
print(training_args)


Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'label': 1, 'idx': 0, 'input_ids': [5122, 2460, 1839, 470, 2822, 428, 3781, 11, 1309, 3436, 262, 1306, 530, 356, 18077, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],


In [8]:
import evaluate

metric = evaluate.load("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
from transformers import AutoModelForSequenceClassification, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer.add_special_tokens({'pad_token': '[PAD]'})
)
trainer.train()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AssertionError: Cannot handle batch sizes > 1 if no padding token is defined.

In [ ]:
trainer.save_model('gpt2_linguistic_fft')
not_acceptable = dataset["validation"][30]

acceptable = dataset["validation"][31]


In [ ]:
from transformers import pipeline

fft_model_name = "gpt2_linguistic_fft"
classifier = pipeline(
    "text-classification", model=gpt2_linguistic_fft
)

In [ ]:
classifier(dataset["validation"][29]['sentence'])
# [{'label': 'LABEL_0', 'score': 0.9874629378318787}]

classifier(dataset["validation"][30]['sentence'])
# [{'label': 'LABEL_1', 'score': 0.9579135775566101}]


In [ ]:
classifier("sea I hill this top")
# [{'label': 'LABEL_0', 'score': 0.9955822825431824}]

classifier("See a hill on top")
# [{'label': 'LABEL_1', 'score': 0.9911277294158936}]

In [ ]:
from transformers import pipeline

fft_model_name = "gpt2_linguistic_fft"
model = pipeline(
    "text-classification", model=fft_model_name
)
answer = model(query)
print(answer)